# Group Comparison analysis pRF propertis 

In [45]:
# Load the necessary libraries 
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os 
import shutil
from matplotlib.backends.backend_pdf import PdfPages

In [46]:
# Define whether you want to run the code on one subject or on the whole dataset

# 1. Run the code on one specific subject 
#subjects = ['sub-22']

# 2. Run the code on glaucoma patients 
#subjects = [f'sub-{i:02}'for i in range(2, 21)]

# 3. Run the code on healthy controls 
subjects = [f'sub-{i:02}'for i in range(2, 47)]

In [50]:
# Define the main variables 
target = 'V3'
source = 'V1'
hemis = ['lh', 'rh']
atlas = 'manual'
delineation = 'manualdelin'
denoising = 'nordic'
task = 'RET'
MAIN_PATH = '/Volumes/FedericaCardillo/pre-processing/projects/EGRET+/derivatives/CFM'
poag = [f'sub-{i:02}' for i in range(2, 21)]
hc = [f'sub-{i:02}' for i in range(21, 47)]
# subjects = poag + hc
visual_areas = ['V2', 'V3', 'V4', 'LO']
freesurfer = f"{MAIN_PATH}/freesurfer"
cumulative = os.path.join(MAIN_PATH,'culumativeplots')
sigmas = []
if os.path.exists(cumulative): 
    shutil.rmtree(cumulative)
os.makedirs(cumulative)
pdf_hemis = PdfPages(os.path.join(cumulative, 'hemis.pdf'))
pdf_hemi = PdfPages(os.path.join(cumulative, 'hemi.pdf'))

tasks = {
    "RET":   {"label": "RET",   "color": {"HC": "blue",      "POAG": "red"}},
    "RET2":  {"label": "RET2",  "color": {"HC": "lightblue", "POAG": "salmon"}},
    "REST":  {"label": "REST",  "color": {"HC": "black",     "POAG": "gray"}}
}
groups = {
    "HC": hc,
    "POAG": poag
}



In [48]:
# PLOT THE CUMULATIVE DISTRIBUTION FUNCTION REPRESENTING THE CONNECTIVE FIELD SIZE SUBJECT PER SUBJECTS

for subj in subjects: 
    sigma = {}

    for hemi in hemis: 
        best_fit_path = f'{MAIN_PATH}/{subj}/ses-1/GM/{hemi}/{denoising}/{task}/{target}/best_fits_prf.csv'
        if not os.path.exists(best_fit_path):
            print(f'Best fit file not found for {subj} ({hemi}).')
            continue

        best_fit_df = pd.read_csv(best_fit_path)
        best_fit_df = best_fit_df[best_fit_df["Best Sigma Finer"] > 0.05]
        best_fit_df = best_fit_df[best_fit_df["Best Variance Explained Finer"] > 0.3]
        sigma[hemi] = best_fit_df["Best Sigma Finer"].values

    # Plot the hemispheres separately 
    fig_one, axs = plt.subplots(1, 2, figsize=(14, 6))
    for i in range(len(hemis)):
        hemi = hemis[i]
        if hemi in sigma:
            axs[i].hist(sigma[hemi], bins=50, cumulative=True, density=True, alpha=0.7)
            axs[i].set_title(f"{subj} - {hemi} - {target}-{source}")
            plt.xlim(0, max(max(sigma[hemi]), 1))
            axs[i].set_xlabel("Connective Field Sigma (degrees)")
            axs[i].set_ylabel("Cumulative Probability")
            axs[i].grid(True)

    fig_one.tight_layout()
    pdf_hemis.savefig(fig_one)
    plt.close(fig_one)

    # Plot the hemispheres together  
    if all(h in sigma for h in hemis):
        sigma_total = list(sigma['lh']) + list(sigma['rh']) 
        fig_two = plt.figure(figsize=(7, 6))
        plt.hist(sigma_total, bins=50, cumulative=True, density=True, alpha=0.7, color='blue')
        plt.title(f"{subj} - {target}-{source}")
        plt.xlim(0, max(sigma_total))
        plt.xlabel("Connective Field Sigma (degrees)")
        plt.ylabel("Cumulative Probability")
        plt.grid(True)
        pdf_hemi.savefig(fig_two) 
        plt.close(fig_two)

pdf_hemis.close()
pdf_hemi.close()

Best fit file not found for sub-03 (lh).
Best fit file not found for sub-03 (rh).
Best fit file not found for sub-24 (lh).
Best fit file not found for sub-24 (rh).


In [52]:
pdf_groupcomparison = PdfPages(os.path.join(cumulative, 'groupcomparison.pdf'))

def sigma(subj, visual_area, task_name):
    sigma_vals = []
    for hemi in hemis:
        path = f'{MAIN_PATH}/{subj}/ses-1/GM/{hemi}/{denoising}/{task_name}/{target}/best_fits_prf.csv'
        if not os.path.exists(path):
            continue
        df = pd.read_csv(path)
        df["Best Sigma Finer"] = pd.to_numeric(df["Best Sigma Finer"], errors='coerce')
        df["Best Variance Explained Finer"] = pd.to_numeric(df["Best Variance Explained Finer"], errors='coerce')
        df = df.dropna(subset=["Best Sigma Finer", "Best Variance Explained Finer"])
        df = df[(df["Best Sigma Finer"] > 0.05) & (df["Best Variance Explained Finer"] > 0.3)]
        sigma_vals.extend(df["Best Sigma Finer"].values)
    return sigma_vals


def individual_cdfs(group, color, label, visual_area, task_name):
    cdfs = []
    included_subjects = 0
    for subj in group:
        sigma_vals = sigma(subj, visual_area, task_name)
        if not sigma_vals:
            continue
        included_subjects += 1
        sorted_sigma = np.sort(sigma_vals)
        cdf = np.linspace(0, 1, len(sorted_sigma))
        plt.plot(sorted_sigma, cdf, linestyle='--', color=color, alpha=0.4)
        grid_cdf = np.interp(x, sorted_sigma, cdf, left=0, right=1)
        cdfs.append(grid_cdf)
    if cdfs:
        median = np.median(cdfs, axis=0)
        plt.plot(x, median, color=color, linewidth=2.5, label=f"{label} Median (N={included_subjects})")
    return cdfs

groupcomp_path = os.path.join(cumulative, 'groupcomparison.pdf')
with PdfPages(groupcomp_path) as pdf_groupcomparison:
    for visual_area in visual_areas:
        x = np.linspace(0, 10, 500)

        # A. Plot individual group/task combinations
        for group_name, group_subjects in groups.items():
            for task_name, task_info in tasks.items():
                plt.figure(figsize=(6, 6))
                cdfs = []
                included_subjects = 0

                for subj in group_subjects:
                    sigma_vals = sigma(subj, visual_area, task_name)
                    if not sigma_vals:
                        continue
                    included_subjects += 1
                    sorted_sigma = np.sort(sigma_vals)
                    cdf = np.linspace(0, 1, len(sorted_sigma))
                    plt.plot(sorted_sigma, cdf, linestyle='--', color=task_info["color"][group_name], alpha=0.4)
                    grid_cdf = np.interp(x, sorted_sigma, cdf, left=0, right=1)
                    cdfs.append(grid_cdf)

                if cdfs:
                    median = np.median(cdfs, axis=0)
                    label = f"{group_name} {task_info['label']} Median (N={included_subjects})"
                    plt.plot(x, median, color=task_info["color"][group_name], linewidth=2.5, label=label)

                plt.title(f"{visual_area} - {group_name} - {task_name}")
                plt.xlabel("Connective Field Sigma (degrees)")
                plt.ylabel("Cumulative Probability")
                plt.grid(True)
                plt.legend()
                pdf_groupcomparison.savefig()
                plt.close()

        # B. Plot all groups/tasks in one comparison
        plt.figure(figsize=(7, 7))
        for group_name, group_subjects in groups.items():
            for task_name, task_info in tasks.items():
                cdfs = []
                included_subjects = 0

                for subj in group_subjects:
                    sigma_vals = sigma(subj, visual_area, task_name)
                    if not sigma_vals:
                        continue
                    included_subjects += 1
                    sorted_sigma = np.sort(sigma_vals)
                    cdf = np.linspace(0, 1, len(sorted_sigma))
                    grid_cdf = np.interp(x, sorted_sigma, cdf, left=0, right=1)
                    cdfs.append(grid_cdf)

                if cdfs:
                    median = np.median(cdfs, axis=0)
                    label = f"{group_name} {task_info['label']} (N={included_subjects})"
                    plt.plot(x, median, color=task_info["color"][group_name], linewidth=2.5, label=label)

        plt.title(f"{visual_area} - All Groups & Tasks Comparison")
        plt.xlabel("Connective Field Sigma (degrees)")
        plt.ylabel("Cumulative Probability")
        plt.grid(True)
        plt.legend()
        pdf_groupcomparison.savefig()
        plt.close()


No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that 